In [2]:
import pandas as pd

In [5]:
df = pd.read_csv('D:/Pepsico/input_data.csv',index_col=False, encoding="ISO-8859-1")

In [6]:
df.head()

,projName,accolNumber,PDA_projName,projType,projDesc,formulaNumber,owner,sector,Unnamed: 8,Unnamed: 9,...,baseProduct,prodModifications,newIngradient,approvedPackage,potentialMicrobial,cross_contactAllergens,chokeHazard,operationalAllergen,Unnamed: 44,Unnamed: 45
0,PRD - Reformulation Sweetcorn 10905989_IFF_SUA,46172,Cheetos Shots Sweetcorn / Cerezza TV Sweetcorn,Productivity,Reformulation of SweetCorn 10905989 seasoning ...,not provided,Zeynep Yolcu 0090 850 279 40 00,ESSA,NaN,NaN,...,No,No,Yes,No,Yes,Yes,No,Yes,NaN,NaN
1,PRD - Reformulation - SYM Peanut Flavour_Turkey,31989,Cheetos Peanut,Productivity,Reformulation of Peanut 355722 seasoning to dr...,not provided,Zeynep Yolcu 0090 850 279 40 00,ESSA,NaN,NaN,...,No,No,Yes,No,needs to be decided as to what value,needs to be decided as to what value,needs to be decided as to what value,Yes,NaN,NaN
2,Chipniks Commercialization at Awesome Snacks,NaN,Simba Chipniks,Capacity Expansion,Leveraging on Awesome Snacks extra fryer capac...,NaN,Sherwin Tlhoaele,ESSA,NaN,NaN,...,No,Yes,No,No,No,Yes,No,NaN,NaN,NaN
3,Crisp-mas 2018,50178,Walkers LTO christmas,Refresh,5 Christmas LTO's 3 x NPD 2 x L&S,christmas,Steve Wilson,ESSA,NaN,NaN,...,No,No,No,No,No,Yes,No,Yes,NaN,NaN
4,Simba Shapes Next Gen,36559,"Simba Shapes - Playz, Simba Shapes Twistz",Breakthrough - South Africa & Nigeria,Purchase an existing Pellet spec used by Pepsi...,unknown,Dina Atoyan (+7(495)9370550 (*63402) ),ESSA,NaN,NaN,...,Yes,Yes,Yes,No,No,Yes,Yes,Yes,NaN,NaN


In [92]:
import os
import re
import nltk
import numpy as np
import string
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.tokenize import RegexpTokenizer

exclude = set(string.punctuation) 
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')
#newStopWords = []
#stop_words.extend(newStopWords)
table = str.maketrans('', '', string.punctuation)
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def normalize_document(doc):
    # tokenize document
    tokens = doc.split()
    # remove punctuation from each word
    tokens = [w.translate(table) for w in tokens]
    # convert to lower case
    lower_tokens = [w.lower() for w in tokens]
    #remove spaces
    stripped = [w.strip() for w in lower_tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter stopwords out of document
    filtered_tokens = [token for token in words if token not in stop_words]
    #apply Stemming
    #stemmed = [porter.stem(word) for word in filtered_tokens]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [93]:
norm_projDesc = normalize_corpus(df['projDesc'])

In [94]:
norm_projDesc[0]

'reformulation sweetcorn seasoning drive seasoning cost per kg reduction deliver productivity product qualification technical match action standart change base product production seasoning changing'

In [17]:
norm_projDesc[1]

'reformulation peanut seasoning drive seasoning cost per kg reduction deliver productivity product qualification technical match action standart change base product production seasoning changing'

In [19]:
df['potentialMicrobial'][1]='Yes'

C:\Users\103467\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
train_label=[]
for i in range(len(df)):
    if (df['potentialMicrobial'][i] =='Yes'):
        train_label.append(1)
    else:
        train_label.append(0)

In [25]:
train_label

[1, 1, 0, 0, 0, 1, 0, 0, 0, 0]

In [81]:
import numpy as np


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# Vectorization parameters

# Range (inclusive) of n-gram sizes for tokenizing text.
NGRAM_RANGE = (1, 2)

# Limit on the number of features. We use the top 20K features.
TOP_K = 20000

# Whether text should be split into word or character n-grams.
# One of 'word', 'char'.
TOKEN_MODE = 'word'

# Minimum document/corpus frequency below which a token will be discarded.
MIN_DOCUMENT_FREQUENCY = 2

# Limit on the length of text sequences. Sequences longer than this
# will be truncated.
MAX_SEQUENCE_LENGTH = 500


def ngram_vectorize(train_texts, train_labels, val_texts):
    """Vectorizes texts as ngram vectors.

    1 text = 1 tf-idf vector the length of vocabulary of uni-grams + bi-grams.

    # Arguments
        train_texts: list, training text strings.
        train_labels: np.ndarray, training labels.
        val_texts: list, validation text strings.

    # Returns
        x_train, x_val: vectorized training and validation texts
    """
    # Create keyword arguments to pass to the 'tf-idf' vectorizer.
    kwargs = {
            'ngram_range': NGRAM_RANGE,  # Use 1-grams + 2-grams.
            'dtype': 'int32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': TOKEN_MODE,  # Split text into word tokens.
            'min_df': MIN_DOCUMENT_FREQUENCY,
    }
    vectorizer = TfidfVectorizer(**kwargs)

    # Learn vocabulary from training texts and vectorize training texts.
    x_train = vectorizer.fit_transform(train_texts)

    # Vectorize validation texts.
    x_val = vectorizer.transform(val_texts)

    # Select top 'k' of the vectorized features -  ignored for now, as we do not have any correlation established
    #selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    #selector.fit(x_train, train_labels)
    #x_train = selector.transform(x_train)
    #x_val = selector.transform(x_val)

    x_train = x_train.astype('float32')
    x_val = x_val.astype('float32')
    return x_train, x_val

In [82]:
# Vectorize texts.
x_train, x_test = ngram_vectorize(norm_projDesc, train_label, ["test"])

C:\Users\103467\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [83]:
x_train.shape

(10, 67)

In [84]:
x_train.T.shape

(67, 10)

In [85]:
X_dense = x_train.toarray()

In [86]:
X_dense[0,:]

array([0.14742255, 0.14742255, 0.        , 0.14742255, 0.14742255,
       0.        , 0.12897731, 0.14742255, 0.14742255, 0.        ,
       0.14742255, 0.14742255, 0.        , 0.        , 0.        ,
       0.14742255, 0.14742255, 0.14742255, 0.14742255, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.14742255, 0.14742255, 0.        , 0.        ,
       0.14742255, 0.14742255, 0.        , 0.        , 0.        ,
       0.        , 0.12897731, 0.14742255, 0.        , 0.        ,
       0.25795463, 0.14742255, 0.14742255, 0.12897731, 0.14742255,
       0.14742255, 0.14742255, 0.        , 0.12897731, 0.14742255,
       0.14742255, 0.14742255, 0.14742255, 0.        , 0.        ,
       0.38693193, 0.14742255, 0.14742255, 0.14742255, 0.        ,
       0.14742255, 0.14742255, 0.14742255, 0.14742255, 0.        ,
       0.        , 0.        ], dtype=float32)

In [64]:
from sklearn.decomposition import PCA
n_components = 2
whiten = False
random_state = 42
svd_solver="full"

In [90]:
pca = PCA(n_components=n_components,svd_solver=svd_solver,whiten=whiten, random_state=42)
X_pca = pca.fit_transform(X_dense)

In [91]:
X_pca.shape

(10, 2)